1) Select any free API (the choice is up to you, you can use any from https://rapidapi.com/collection/list-of-free-apis).
2) Create the database which stores the infomration related to the API chosen (at least 2 tables).
3) In Python, 3.1) implement the logic which fetches the data from API and moves it to Database 3.2) Fetches the data from database and 3.2.1) If there is the API functionality
(POST/GET request) which allows you to use the data from database in an API call, do it 3.2.2.) If not - display the data from database as Pandas dataframe

In [32]:
import json
import requests
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [33]:
Base = declarative_base()

class Breed(Base):
    __tablename__ = 'Breeds'
    id = Column(Integer, primary_key=True)
    breed = Column(String(30))
    origin = Column(String(50))

class BreedImage(Base):
    __tablename__ = 'Breeds_img'
    id = Column(Integer, primary_key=True)
    url = Column(String(50))
    img = Column(String(50))

engine = create_engine('mysql+pymysql://root@localhost/test',  pool_recycle=3600)
Session = sessionmaker(bind=engine)
session = Session()

Base.metadata.create_all(engine)


C:\Users\dovil\AppData\Local\Temp\ipykernel_15652\3707988397.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [34]:

url = "https://dog-breeds2.p.rapidapi.com/dog_breeds/group/breed_status/extinct"
headers = {
    "X-RapidAPI-Key": "51b07a06c6mshbaae7fd65806f5cp13da34jsn59c5aa35baa7",
    "X-RapidAPI-Host": "dog-breeds2.p.rapidapi.com"
}

response = requests.get(url, headers=headers)
api_data = response.json()

for record in api_data:
    breed = Breed(id=record['id'], breed=record['breed'], origin=record['origin'])
    session.add(breed)
    breed_image = BreedImage(id=record['id'], url=record['url'], img=record['img'])
    session.add(breed_image)

session.commit()

In [35]:
breeds = session.query(Breed.breed, Breed.origin, BreedImage.url, BreedImage.img).\
    join(BreedImage, Breed.id == BreedImage.id).all()

session.close()

import pandas as pd
df = pd.DataFrame(breeds, columns=['Breed', 'Origin', 'Link', 'Img'])
df.head()

,Breed,Origin,Link,Img
0,Old English Terrier,England,https://en.wikipedia.org/wiki/Old_English_Terrier,https://upload.wikimedia.org/wikipedia/commons...
1,African hairless dog,Africa,https://en.wikipedia.org/wiki/Abyssinian_sand_...,https://upload.wikimedia.org/wikipedia/commons...
2,Alaunt,Pontic-Caspian Steppe,https://en.wikipedia.org/wiki/Alaunt,https://upload.wikimedia.org/wikipedia/commons...
3,Alpine Mastiff,Swiss and Italian Alps,https://en.wikipedia.org/wiki/Alpine_Mastiff,https://upload.wikimedia.org/wikipedia/commons...
4,Belgian Mastiff,Belgium and the Netherlands,https://en.wikipedia.org/wiki/Belgian_Mastiff,https://upload.wikimedia.org/wikipedia/commons...
